In [2]:
import pandas as pd
import numpy as np

In [3]:
data_dir = "./data/raw/combined_data_1.txt"
data = []

with open(data_dir, "r") as file:
    movie_id = None
    for line in file:
        line = line.strip()
        if line.endswith(":"):
            movie_id = int(line[:-1])
        else:
            customer_id, rating, date = line.split(",")
            data.append([movie_id, int(customer_id), int(rating), date])

data = pd.DataFrame(data, columns=["itemID", "userID", "rating", "timestamp"])

# convert date to timestamp
data["timestamp"] = pd.to_datetime(data["timestamp"]).apply(lambda x: x.timestamp())

# reindex IDs
user_id = data[["userID"]].drop_duplicates().reindex()
user_id["userId"] = np.arange(len(user_id))
data = pd.merge(data, user_id, on=["userID"], how="left")

item_id = data[["itemID"]].drop_duplicates().reindex()
item_id["itemId"] = np.arange(len(item_id))
data = pd.merge(data, item_id, on=["itemID"], how="left")

data = data.drop(columns=["userID", "itemID"])

In [4]:
# TODO remove users with less than 5 interactions

# reduce rows
NUM_USERS = 5000
subset_user_ids = data["userId"].unique()[:NUM_USERS]
data = data[data["userId"].isin(subset_user_ids)]

print(data.head())
print("Range of userId is [{}, {}]".format(data.userId.min(), data.userId.max()))
print("Range of itemId is [{}, {}]".format(data.itemId.min(), data.itemId.max()))
print("Range of rating is [{}, {}]".format(data.rating.min(), data.rating.max()))

data.to_csv("./data/processed/combined1.csv")

   rating     timestamp  userId  itemId
0       3  1.125965e+09       0       0
1       5  1.115942e+09       1       0
2       4  1.129680e+09       2       0
3       4  1.135555e+09       3       0
4       3  1.083542e+09       4       0
Range of userId is [0, 4999]
Range of itemId is [0, 4498]
Range of rating is [1, 5]
